In [1]:
from paz.models import SSD300
import cv2
import time
import numpy as np
from paz.pipelines import PreprocessImage, PreprocessBoxes
from paz.models.detection.utils import create_prior_boxes
import os
from paz.evaluation import evaluateMAP
from paz.evaluation.detection import calculate_relevance_metrics, compute_matches, calculate_average_precisions
from paz.datasets import VOC, get_class_names
from paz.pipelines import DetectSingleShot
from paz.backend.image import load_image

In [2]:
class_names = ['N/A', 'Wolf', 'Dog']

data_manager_train = VOC('voc_dataset', 'train', class_names, name='VOC2007', evaluate=True)
dataset_train = data_manager_train.load_data()

data_manager_test = VOC('voc_dataset', 'test', class_names, name='VOC2007', evaluate=True)
dataset_test = data_manager_test.load_data()

In [3]:
score_thresh, nms_thresh = 0.01, .45

model = SSD300(3, base_weights=None, head_weights=None)
model.load_weights('object_detection/trained_models/SSD300/weights.hdf5')
detector = DetectSingleShot(model, class_names, score_thresh, nms_thresh)

In [4]:
class_to_arg = {class_name:count for class_name, count in zip(data_manager_train.class_names, range(len(data_manager_train.class_names)))}

In [5]:
def get_metrics(dataset, detector, class_to_arg):
    num_positives, score, match = compute_matches(dataset, detector, class_to_arg)
    precision, recall = calculate_relevance_metrics(num_positives, score, match)
    average_precisions = calculate_average_precisions(precision, recall)
    
    total_recall = 0
    count_recall = 0
    for r in recall:
        if r is not None:
            total_recall += r.mean()
            count_recall += 1
    final_recall = total_recall / count_recall
           
    average_precisions = calculate_average_precisions(precision, recall)
    final_precision = np.nanmean(average_precisions)
    
    maps = []
    aps = []
    for iou in np.arange(0.5,1,0.05):
        map_iou = evaluateMAP(detector, dataset, class_to_arg, iou_thresh=iou)
        maps.append(map_iou['map'])
        aps.append(map_iou['ap'])
        print(maps)
    
    return final_precision, final_recall, maps[0], np.array(maps).mean()

In [6]:
precision, recall, map_voc, map_coco = get_metrics(dataset_test, detector, class_to_arg)

1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step
[0.8548970196982886]
1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 19ms/step
[0.8548970196982886, 0.844881709408017]
1/1 [==============================] - 0s 19ms/step


...

1/1 [==============================] - 0s 18ms/step
[0.8548970196982886, 0.844881709408017, 0.8247681160762873, 0.7866640393827232, 0.647317653062633, 0.481705799700621, 0.257967425052428]
1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 16ms/step
[0.8548970196982886, 0.844881709408017, 0.8247681160762873, 0.7866640393827232, 0.647317653062633, 0.481705799700621, 0.257967425052428, 0.07597769675514984]
1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step
[0.8548970196982886, 0.844881709408017, 0.8247681160762873, 0.7866640393827232, 0.647317653062633, 0.481705799700621, 0.257967425052428, 0.07597769675514984, 0.012893788592976064]
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 18ms/step
[0.8548970196982886, 0.844881709408017, 0.8247681160762873, 0.7866640393827232, 0.647317653062633, 0.481705799700621, 0.257967425052428, 0.07597769675514984, 0.012893788592976064, 0.0]


In [7]:
precision, recall, map_voc, map_coco

(0.8548970196982886,
 0.9333453503938184,
 0.8548970196982886,
 0.4787073247729124)

In [8]:
precision, recall, map_voc, map_coco = get_metrics(dataset_train, detector, class_to_arg)

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step
[0.9290612511492218, 0.9177941675272024, 0.9067567012753545, 0.8867442389256537, 0.8654053690040022, 0.8002805431437271, 0.6958653904754732, 0.42363519450626586, 0.07530011783281133, 0.0017895779733525887]


In [9]:
precision, recall, map_voc, map_coco

(0.9290612511492218,
 0.9430794687588164,
 0.9290612511492218,
 0.6502632551813065)